# Segmentation & Clustering Neighbor hoods in Toronto
## Coursera Capstone Project 

### Yvel MARCELIN


### Part 1 - Converting Wikipedia html table into a DataFrame

In [120]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as Bs
import requests as req

In [121]:
# get the html content page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_age = req.get(url).text
# use Beautiful soup to parse the html page
bs = Bs(html_age,'html.parser')
#get code postal html table
tables = bs.table
# create the code postal dataframe
df_canada = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])
for tr_cell in tables.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df_canada.loc[len(df_canada)] = row_data
# top five rows in the dataframe
print(df_canada.shape)
df_canada.head()

(287, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [122]:
postalcodes = df_canada['PostalCode'].nunique()
neighborhoods= df_canada['Neighborhood'].nunique()
boroughs = df_canada['Borough'].nunique()
print('Unique Postalcodes : ' + str(postalcodes))
print('Unique Boroughs  : '+ str(boroughs))
print('Unique Neighborhoods  :' + str(neighborhoods))

Unique Postalcodes : 180
Unique Boroughs  : 12
Unique Neighborhoods  :208


- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [123]:
df_canada = df_canada.loc[df_canada.Borough != 'Not assigned']
print(df_canada.shape)
df_canada.head()

(210, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [124]:
postalcodes = df_canada['PostalCode'].nunique()
neighborhoods= df_canada['Neighborhood'].nunique()
boroughs = df_canada['Borough'].nunique()
print('Unique Postalcodes : ' + str(postalcodes))
print('Unique Boroughs  : '+ str(boroughs))
print('Unique Neighborhoods  :' + str(neighborhoods))

Unique Postalcodes : 103
Unique Boroughs  : 11
Unique Neighborhoods  :208


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [125]:
df_canada=df_canada.groupby(['PostalCode','Borough'],)['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df_canada=df_canada.reset_index(drop=False)
print(df_canada.shape)
df_canada.head(10)

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [126]:
 df_canada.loc[df_canada.Neighborhood == 'Not assigned']

,PostalCode,Borough,Neighborhood
93,M9A,Queen's Park,Not assigned


In [127]:
df_canada.loc[df_canada['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_canada['Borough']
df_canada.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [128]:
 df_canada.loc[df_canada.Borough == "Queen's Park"]

,PostalCode,Borough,Neighborhood
93,M9A,Queen's Park,Queen's Park


In [129]:
postalcodes = df_canada['PostalCode'].nunique()
neighborhoods= df_canada['Neighborhood'].nunique()
boroughs = df_canada['Borough'].nunique()
print('Unique Postalcodes : ' + str(postalcodes))
print('Unique Boroughs  : '+ str(boroughs))
print('Unique Clustering Neighborhoods  :' + str(neighborhoods))

Unique Postalcodes : 103
Unique Boroughs  : 11
Unique Clustering Neighborhoods  :102


In [130]:
ids = df_canada.Neighborhood
df_canada[ids.isin(ids[ids.duplicated()])].sort_values("Neighborhood")

,PostalCode,Borough,Neighborhood
85,M7A,Downtown Toronto,Queen's Park
93,M9A,Queen's Park,Queen's Park


## PART 2: 
### Now, let's get the Latitude and Longitude coordenates for every PostalCode and add those columns to the dataframe

#### I try to use Geocoder but it still not works well.Let's using that link 'https://cocl.us/Geospatial_data'

In [131]:
can_coord = pd.read_csv('https://cocl.us/Geospatial_data')
can_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [132]:
df_canada = pd.merge(df_canada,can_coord,left_on='PostalCode', right_on='Postal Code')
df_canada.drop(columns =['Postal Code'],axis =1, inplace =True)

In [133]:
df_canada.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
